Before starting this cell, create a virtual environment and install the requiremnts.txt file and choose the evironment's python as kernel for this notebook.

First of all, I am sorting the articles based on the relatibility of topic with Hamas and israel. 
This below cell is executing a script which saves 'relevant_articles.json' file (it contains 24k articles now)

In [ ]:
import json
import re

# Load the JSON file
with open('news_article.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def clean_text(text):
    # Remove special characters, punctuation, and extra whitespace
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.lower() 

relevant_articles = []
for article in data:
    if 'Israel' in article['articleBody'] and 'Hamas' in article['articleBody']:
        relevant_articles.append({
            'title': article['title'],
            'source': article['source'],
            'articleBody': clean_text(article['articleBody'])
        })  

file_path = 'relevant_articles.json'

# Write the list to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(relevant_articles, json_file, indent=4)

print(f"Relevant articles have been saved to {file_path}")        


The below cell defines a function which picks keywords from the question asked.

In [1]:
#Key word extraction programm

stopwords = {
    'the', 'is', 'are', 'and', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'a', 'an',
    'what', 'happened', 'who', 'they', 'did', 'has', 'have', 'had', 'where', 'why',
    'which', 'when', 'it', 'its', 'his', 'her', 'their', 'there', 'here', 'that', 
    'this', 'these', 'those', 'be', 'been', 'being', 'was', 'were', 'would', 
    'should', 'could', 'may', 'might', 'can', 'will', 'shall', 'must', 'do', 
    'does', 'doing', 'done', 'get', 'gets', 'getting', 'got', 'use', 'uses', 
    'using', 'used', 'go', 'goes', 'going', 'gone', 'went', 'take', 'takes', 
    'taking', 'took', 'come', 'comes', 'coming', 'came', 'say', 'says', 
    'saying', 'think', 'thinks', 'thinking', 'thought', 'know', 'knows', 
    'knowing', 'known', 'want', 'wants', 'wanting', 'wanted', 'like', 'likes', 
    'liking', 'liked', 'love', 'loves', 'loving', 'loved', 'need', 'needs', 
    'needing', 'needed', 'try', 'tries', 'trying', 'tried', 'look', 'looks', 
    'looking', 'looked', 'work', 'works', 'working', 'worked', 'help', 'helps', 
    'helping', 'helped', 'ask', 'asks', 'asking', 'asked', 'give', 'gives', 
    'giving', 'given', 'find', 'finds', 'finding', 'found', 'mean', 'means', 
    'meaning', 'meant', 'include', 'includes', 'including', 'included', 
    'become', 'becomes', 'becoming', 'became', 'seem', 'seems', 'seeming', 
    'seemed', 'appear', 'appears', 'appearing', 'appeared', 'contain', 
    'contains', 'containing', 'contained', 'believe', 'believes', 'believing', 
    'believed', 'represent', 'represents', 'representing', 'represented', 
    'understand', 'understands', 'understanding', 'understood', 'ensure', 
    'ensures', 'ensuring', 'ensured', 'require', 'requires', 'requiring', 
    'required', 'consider', 'considers', 'considering', 'considered', 'over', 'major', 'main', 'primarily', 'fought', 'fight', 
    'fighting', 'result', 'war', 'conflict', 'sought', 'lead', 
}



def extract_keywords(text):
    # Define a regular expression pattern to match words
    pattern = r'\b\w+\b'
    # Find all words in the text using the regular expression
    words = re.findall(pattern, text.lower())
    # Define a set of stopwords to filter out common words
    
    # Filter out stopwords and return the remaining words as keywords
    keywords = [word for word in words if word not in stopwords]
    return keywords



Below cell takes question as input and extract keywords from it and filter out articles which has all these keywords in it and saves in 'key_extr_articles' variable

In [3]:
#cutting down articles via finding keywords

question = "Why israeli military has been hitting gaza with airstrikes?"
# question = 'When Hamas attacked on Israel?'
keywords = extract_keywords(question)
print(keywords)

key_extr_articles = []
for article in relevant_articles:
    all_keywords_present = all(keyword in article['articleBody'] for keyword in keywords)
    if all_keywords_present:
        key_extr_articles.append({
            'title': article['title'],
            'source': article['source'],
            'articleBody': clean_text(article['articleBody'])
        })

['israeli', 'military', 'hitting', 'gaza', 'airstrikes']


In [ ]:

print('We can see the number of useful articles has already been reduced to ')
print((key_extr_articles))

85

Below cell defines a mmr(Maximal Marginal Relevance) function whcih is useful for discriminating articles further.

In [5]:
#mmr program


import json
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load articles from JSON file
def load_articles(key_articles):
    
    return [article['articleBody'] for article in key_articles]

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to get embeddings
def get_embeddings(texts):
    return model.encode(texts, convert_to_tensor=True)

# MMR Algorithm
def mmr(query_embedding, document_embeddings, documents, top_k=3, lambda_param=0.5):
    query_embedding = query_embedding.cpu().detach().numpy()
    document_embeddings = document_embeddings.cpu().detach().numpy()
    
    # Calculate cosine similarity between query and documents
    query_doc_similarity = cosine_similarity(query_embedding, document_embeddings).flatten()
    
    # Initialize the selected documents list and remaining documents set
    selected_documents = []
    remaining_documents = list(range(len(documents)))
    
    # Select the most relevant document first
    selected_documents.append(np.argmax(query_doc_similarity))
    remaining_documents.remove(selected_documents[-1])
    
    # Select the remaining top_k - 1 documents
    for _ in range(top_k - 1):
        mmr_score = []
        
        for doc_index in remaining_documents:
            # Calculate the diversity term (minimum similarity to already selected documents)
            diversity = min(cosine_similarity(document_embeddings[doc_index].reshape(1, -1), 
                                              document_embeddings[selected].reshape(1, -1)).flatten()[0] 
                            for selected in selected_documents)
            
            # Calculate the MMR score
            score = (lambda_param * query_doc_similarity[doc_index]) - ((1 - lambda_param) * diversity)
            mmr_score.append((doc_index, score))
        
        # Select the document with the highest MMR score
        selected_doc = max(mmr_score, key=lambda x: x[1])[0]
        selected_documents.append(selected_doc)
        remaining_documents.remove(selected_doc)
    
    return [documents[i] for i in selected_documents]

e:\Assignment\myenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
e:\Assignment\myenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In the below cell, I have defined a threshold limit of 3 articles in the mmr function, means we will get top 3 relevant articles related to question. 

In [7]:
articles = load_articles(key_extr_articles)
    
    # Get embeddings
question_embedding = get_embeddings([question])
article_embeddings = get_embeddings(articles)
    
    # Apply MMR
top_k = 3
selected_articles = mmr(question_embedding, article_embeddings, articles, top_k=top_k)


Article 1:
israel launching a ground offensive in the gaza strip is a certainty but what remains to be seen is how the israeli military goes about it according to a russian analyst there are different ways that any force could employ a ground military operation taking into consideration various aspects boris dolgov a researcher at the russian centre for arab and islamic studies at the institute of oriental studies in moscow told anadolu over the past two weeks israeli officials have repeatedly hinted at launching a ground offensive in gaza all while heavily bombarding the besieged palestinian territory the relentless israeli attacks have now killed more than 5000 palestinians among them over 2000 children and 1100 women the number of people injured in gaza is now over 15200 according to official figures read israel think tank linked to netanyahu promotes unique opportunity to ethnically cleanse gaza israel has targeted all areas of the gaza strip hitting densely populated residential a

Below, defining BERT model to answer the question from choosen top 3 articles

In [8]:
#model

from transformers import BertTokenizer, BertForQuestionAnswering
import torch
import json

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

# Load the JSON file containing the articles

# Define a function to get answers to questions
def get_answer1(question, context, max_length=512):
    answers = []
    for i in range(0, len(context), max_length):
        inputs = tokenizer(question, context[i:i+max_length], return_tensors='pt', truncation=True)
        outputs = model(**inputs)
        start_positions = torch.argmax(outputs.start_logits)
        end_positions = torch.argmax(outputs.end_logits)
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_positions:end_positions+1]))
        answers.append(answer)
    return ' '.join(answers)

'israel launching a ground offensive in the gaza strip is a certainty but what remains to be seen is how the israeli military goes about it according to a russian analyst there are different ways that any force could employ a ground military operation taking into consideration various aspects boris dolgov a researcher at the russian centre for arab and islamic studies at the institute of oriental studies in moscow told anadolu over the past two weeks israeli officials have repeatedly hinted at launching a ground offensive in gaza all while heavily bombarding the besieged palestinian territory the relentless israeli attacks have now killed more than 5000 palestinians among them over 2000 children and 1100 women the number of people injured in gaza is now over 15200 according to official figures read israel think tank linked to netanyahu promotes unique opportunity to ethnically cleanse gaza israel has targeted all areas of the gaza strip hitting densely populated residential areas hospi

Grammar may seems odd in the answer, thats why we have a grammar correction function defined below

In [9]:
#grammar correct 

from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load the pre-trained T5 model and tokenizer for grammar correction
def load_t5_model():
    model_name = 'vennify/t5-base-grammar-correction'
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    return model, tokenizer

# Define a function for grammar correction using T5
def correct_grammar(text, model, tokenizer):
    input_text = f"fix: {text}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    outputs = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text


Finally, we are getting top three answers from the 37k articles, below. 

In [10]:
mod, tok = load_t5_model()
c = 1
for blog in selected_articles:
    print('Answer ', c)
    ans = get_answer1(question=question,context=blog)
    print(correct_grammar(ans, mod, tok))
    c = c+1

e:\Assignment\myenv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91786\.cache\huggingface\hub\models--vennify--t5-base-grammar-correction. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>